In [0]:
import os, uuid
import time, datetime
import multiprocessing
from multiprocessing import cpu_count
import pandas as pd
# from azure.cosmos import CosmosClient

In [0]:
# 储存账户名称
storage_account_name = ""
container_name=""
storage_account_access_key = ""
connect_str = ""

file_location = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/"
# 写入ETL的时间
filename = '/dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/CameraStatus/ETL_date.txt' 
spark.conf.set(
  "fs.azure.account.key."+storage_account_name+".blob.core.windows.net",
  storage_account_access_key)

In [0]:
def gen_dates(b_date, days):
    day = datetime.timedelta(days=1)
    for i in range(days):
        yield b_date + day*i


def get_date_list(start=None, end=None):
    """
    获取日期列表
    :param start: 开始日期
    :param end: 结束日期
    :return:
    """
    if start is None:
        start = datetime.datetime.strptime("2020/12/01", "%Y/%m/%d")
    if end is None:
        end = datetime.datetime.now()
    data = []
    for d in gen_dates(start, (end-start).days):
        data.append(d)
    data_new = []
    for i in data:
      new = str(i)[:10]
      new_time = new.split('-')
      year , month , day = int(new_time[0]) , int(new_time[1]) , int(new_time[2]) 
      date = str(year) + '/' + str(month) + '/' +str(day) 
      data_new.append(date)
    return data_new

data = get_date_list()
with open(filename,'r') as f: 
  ETL_date = f.read()
time = data[data.index(ETL_date)+1]
time

Out[137]: '2021/4/14'

In [0]:
spark.catalog.setCurrentDatabase("camera")

In [0]:
year , month , day = time.split('/')
year , month , day = int(year) , int(month) , int(day)
cameras = dbutils.fs.ls(file_location)
data = [] # 'camera_name' , 'year' , 'month' , 'day' , 'up_load_image_num','positive','negative'
for camera in cameras:
  camera_name = camera[1]
  img_location = file_location + camera_name  + time # wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/00000000000000T/
  camera_name = camera_name.replace('/','')
  try:
    img = dbutils.fs.ls(img_location)
  except:
    print(img_location +" does not exist")
    sql = "insert into camerastatus values ('%s',%d,%d,%d,%d,%d,%d)" % (camera_name,year,month,day,0,0,0)
  else:
    positive , negative = 0 , 0
    df = spark.read.format("json").load(img_location).toPandas()
    for i in df['Messages']:
      i = str(i).split(',')
      score = int(i[-2].replace(')','').replace('Score=',''))
      if score < 128:
        negative += 1
      else:
        positive += 1
    sql = "insert into camerastatus values ('%s',%d,%d,%d,%d,%d,%d)" % (camera_name,year,month,day,len(img),positive,negative)
  spark.sql(sql)
  print(sql)
with open(filename,'w') as f:
  f.write(time)
print(time)

wasbs://espdata@storageaccountcamera.blob.core.windows.net/04670119080008/2021/4/14 does not exist
insert into camerastatus values ('04670119080008',2021,4,14,0,0,0)

In [0]:
%sql
select * from camerastatus

camera_name,year,month,day,up_load_image_num,positive,negative
60210117000118,2021,4,13,4341,2609,1732
04670119080008,2021,4,13,0,0,0
60210117000009,2021,4,12,3555,18,3537
04670119080008,2021,4,12,0,0,0
04670119080008,2021,4,12,0,0,0
60210117000009,2021,4,13,8586,2522,6064
60210117000118,2021,4,12,1145,374,771
LDCSH040,2021,4,12,0,0,0
LDCSH040,2021,4,13,0,0,0


In [0]:
# spark_df.write.format("parquet").mode('append').save("cameraStatus_yesterday")

In [0]:
# def read(): 
#   data = spark.read.format("json").load("wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/0000000000EB9B/2021/4/6/")
#   data = data.toPandas()
#   return data
# for i in range(1):
# #   image_path = location_path +"/" + i[1]
#   p.apply_async(read())
#   display(data)